In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import datetime

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/level2_objectdetection-cv-11/configs/faster_rcnn/RandomBrightness.py')

root='/opt/ml/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#cfg.data.train.ann_file = root + 'pseudo_concat_new.json'
cfg.data.train.ann_file = root + '3___train_MultiStfKFold.json' # train json 정보

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + '3___val_MultiStfKFold.json'

'''
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type = 'Albu',
         transforms = [dict(type='Rotate', p=1, rotate_method='ellipse', limit=15)],
         #transforms = [dict(type='RandomRotate90', p=1)],
         #transforms = [],
         bbox_params = dict(type='BboxParams', format='pascal_voc', label_fields=['gt_labels']),
         keymap = {'img': 'image', 'gt_bboxes': 'bboxes'}),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])]
'''

cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

now = (datetime.datetime.now().replace(microsecond=0) + datetime.timedelta(hours=9)).strftime("%m-%d %H:%M")
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={
            'project': 'trash_detection',
            'entity': 'doyunkim',
            'name' : f'{cfg.model.type}_{now}_dyk',
            'tags': [cfg.model.backbone.type, cfg.model.neck.type]
            },
         interval=10)]

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
#cfg.data.val.pipeline = cfg.data.test.pipeline


cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'+now


for each_head in cfg.model.roi_head.bbox_head:
    if hasattr(each_head, "num_classes"):
        each_head.num_classes = 10 

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train.dataset)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [4]:
datasets[0]


CocoDataset Train dataset with number of images 3906, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3154  | 1 [Paper]     | 4918  | 2 [Paper pack]  | 675   | 3 [Metal]   | 722   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2354  | 6 [Styrofoam] | 976   | 7 [Plastic bag] | 4094  | 8 [Battery] | 132   | 9 [Clothing] | 394   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
model = build_detector(cfg.model)
model.init_weights()

2023-05-15 03:25:44,147 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window12_384_22kto1k.pth
2023-05-15 03:25:44,524 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-15 03:25:44,550 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2023-05-15 03:25:44,560 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2023-05-15 03:25:44,687 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}

In [6]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2023-05-15 03:25:52,231 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-15 03:25:53,782 - mmdet - INFO - Start running, host: root@1c7867573923, work_dir: /opt/ml/level2_objectdetection-cv-11/tools/work_dirs/faster_rcnn_r50_fpn_1x_trash05-15 12:25
2023-05-15 03:25:53,785 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -----------

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:226: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junha4304. Use `wandb login --relogin` to force relogin


Problem at: /opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/logger/wandb.py 97 before_run


wandb: ERROR Error while calling W&B API: project not found (<Response [404]>)


CommError: It appears that you do not have permission to access the requested resource. Please reach out to the project owner to grant you access. If you have the correct permissions, verify that there are no issues with your networking setup.(Error 404: Not Found)